# Big Data... no big deal

# PySpark

![PyOhio](images/logos/pyohio-2020-square-transparent-400.png)

### gh: catherinedevlin/talks

![database logos: PostgreSQL, MySQL, SQLite, dbm](images/databases-slide.png)

![big data](images/big-data-slide.png)

![kiddie table](images/kids-table.jpg)

![map-reduce](images/map-reduce-slide.png)

![kiddie table](images/kids-table.jpg)

![Disney+: Hamilton](images/dss.png)

![hadoop, redis, cassandra, Spark, PySpark](images/big-data-tech-slide.png)

# big data == distributed data

[WaPo police shootings database](https://github.com/washingtonpost/data-police-shootings)


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').getOrCreate()
import databricks.koalas as ksa

temp_df = spark.read.csv(
    "data-source/fatal-police-shootings-data.csv", header=True, 
)
!rm -rf my-cloud-service/shootings
temp_df.write.partitionBy("state").format("parquet").save("my-cloud-service/shootings")
# Also available: JSON, AVRO, ORC, Delta, ...

In [2]:
ls my-cloud-service/shootings/

'state=AK'/  'state=FL'/  'state=LA'/  'state=NC'/  'state=OK'/  'state=VA'/
'state=AL'/  'state=GA'/  'state=MA'/  'state=ND'/  'state=OR'/  'state=VT'/
'state=AR'/  'state=HI'/  'state=MD'/  'state=NE'/  'state=PA'/  'state=WA'/
'state=AZ'/  'state=IA'/  'state=ME'/  'state=NH'/  'state=RI'/  'state=WI'/
'state=CA'/  'state=ID'/  'state=MI'/  'state=NJ'/  'state=SC'/  'state=WV'/
'state=CO'/  'state=IL'/  'state=MN'/  'state=NM'/  'state=SD'/  'state=WY'/
'state=CT'/  'state=IN'/  'state=MO'/  'state=NV'/  'state=TN'/   _SUCCESS
'state=DC'/  'state=KS'/  'state=MS'/  'state=NY'/  'state=TX'/
'state=DE'/  'state=KY'/  'state=MT'/  'state=OH'/  'state=UT'/


In [3]:
ls -lh my-cloud-service/shootings/state=OH

total 12K
-rw-r--r-- 1 jovyan users 9.0K Jul 19 21:08 part-00000-93ad549a-49cf-4f8d-b073-b96ac76abdbf.c000.snappy.parquet


In [4]:
df = spark.read.parquet("my-cloud-service/shootings")

In [6]:
df.show()

+---+--------------------+----------+----------------+----------+---+------+----+--------------+-----------------------+------------+-----------+-----------+-----+
| id|                name|      date| manner_of_death|     armed|age|gender|race|          city|signs_of_mental_illness|threat_level|       flee|body_camera|state|
+---+--------------------+----------+----------------+----------+---+------+----+--------------+-----------------------+------------+-----------+-----------+-----+
|  8|     Matthew Hoffman|2015-01-04|            shot|toy weapon| 32|     M|   W| San Francisco|                   True|      attack|Not fleeing|      False|   CA|
| 19|      Patrick Wetter|2015-01-06|shot and Tasered|     knife| 25|     M|   W|      Stockton|                  False|      attack|Not fleeing|      False|   CA|
| 43|      Brian  Barbosa|2015-01-11|            shot|       gun| 23|     M|   H|    South Gate|                  False|      attack|Not fleeing|      False|   CA|
| 63|           

In [15]:
df.select('name', 'date', 'age', 'city').filter(df.state=='OH').show()

+--------------------+----------+---+-----------+
|                name|      date|age|       city|
+--------------------+----------+---+-----------+
|Hashim Hanif Ibn ...|2015-01-07| 41|   Columbus|
|   Jermonte Fletcher|2015-01-27| 33|   Columbus|
|    Theodore Johnson|2015-03-10| 64|  Cleveland|
|       Brandon Jones|2015-03-19| 18|  Cleveland|
|         Jason Smith|2015-03-21| 42|   Columbus|
|       Robert Rooker|2015-03-29| 26|     Jasper|
|        David Parker|2015-04-28| 58|  Mansfield|
|    Jonathan  Colley|2015-05-20| 52|      Green|
|      Jeremy Linhart|2015-06-09| 30|    Findlay|
|    QuanDavier Hicks|2015-06-09| 22| Cincinnati|
|   Trepierre Hummons|2015-06-19| 21| Cincinnati|
|    Charles Crandall|2015-07-14| 76|Southington|
|       Samuel DuBose|2015-07-19| 43| Mt. Auburn|
|    Dontae L. Martin|2015-07-23| 34|     Dayton|
|   Reginald Marshall|2015-08-12| 27|     Toledo|
|        Tyler Gerken|2015-08-20| 19|    Beverly|
| Charles Robert Shaw|2015-09-01| 76|  Twinsburg|


In [8]:
df.describe('age').show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|              5231|
|   mean| 37.13152360925253|
| stddev|13.127085188139786|
|    min|                12|
|    max|                91|
+-------+------------------+



In [9]:
by_city = df.filter(df.state=='OH').groupby('city').count().withColumnRenamed('count', 'n')
by_city.orderBy(by_city.n.desc()).show()

+------------------+---+
|              city|  n|
+------------------+---+
|          Columbus| 31|
|         Cleveland|  7|
|             Akron|  7|
|        Cincinnati|  7|
|            Toledo|  5|
|            Dayton|  5|
|         Kettering|  3|
|         Mansfield|  3|
|             Parma|  3|
| Sylvania Township|  2|
|       Springfield|  2|
|          Hamilton|  2|
|              Kent|  2|
|  Madison Township|  2|
|         Fairfield|  2|
|Ashtabula Township|  1|
|              Lima|  1|
|          Oak Hill|  1|
|            Girard|  1|
|        Willoughby|  1|
+------------------+---+
only showing top 20 rows



In [10]:
df.createOrReplaceTempView("shootings")

QRY = """
SELECT name, age, signs_of_mental_illness 
FROM   shootings 
WHERE  state='OH' 
AND    date >= '2020-01-01'  """
oh2020 = spark.sql(QRY)
oh2020.show()

+--------------------+---+-----------------------+
|                name|age|signs_of_mental_illness|
+--------------------+---+-----------------------+
|Joshua Franklin R...| 41|                  False|
|  Joshua James Brown| 34|                  False|
|        Chad Nicolia| 41|                  False|
|    Abdirahman Salad| 15|                  False|
|       Joseph Jewell| 17|                  False|
|       William Resto| 54|                  False|
|      Tyler M. Jones| 24|                  False|
|          Gary Brown| 43|                  False|
|      Robert Tincher| 49|                   True|
|            John Vik| 47|                  False|
|         James Tober| 68|                   True|
|Terena Nicole Thu...| 36|                  False|
+--------------------+---+-----------------------+



In [11]:
kdf = df.to_koalas()

In [12]:
kdf.state.value_counts()

CA    807
TX    486
FL    355
AZ    256
CO    201
GA    185
OK    165
OH    157
NC    154
WA    152
TN    143
MO    143
PA    111
LA    110
NM    105
IL    104
AL    104
NY    102
NV     98
IN     97
KY     97
VA     95
WI     93
SC     89
OR     87
AR     83
MI     80
MD     79
NJ     68
MS     65
MN     61
UT     61
WV     54
KS     50
ID     42
AK     39
MA     35
MT     33
IA     31
HI     30
NE     24
ME     22
CT     21
SD     17
WY     14
DC     13
DE     13
NH     13
ND     11
VT      9
RI      4
Name: state, dtype: int64

### gh: catherinedevlin/talks

"The Kids' Table" by Clover_1 is licensed under CC BY-NC 2.0. To view a copy of this license, visit https://creativecommons.org/licenses/by-nc/2.0/